In [1]:
pip install imblearn

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 6.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.multiclass import OneVsOneClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer, KNNImputer

s3= boto3.resource('s3')
bucket_name= 'morgan-gant-data448-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'framingham.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [8]:
#Defining input and target variables
x = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
y= heart['TenYearCHD']

#Splitting the data
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size= .2, stratify=y)

In [14]:
#Defining the imputer
imputer= KNNImputer(n_neighbors= 5, weights= 'distance').fit(x_train)

#Imputing missing values
x_train_imp= pd.DataFrame(imputer.transform(x_train), columns= x_train.columns)
x_test_imp= pd.DataFrame(imputer.transform(x_test), columns= x_test.columns)

Random Forest

In [15]:
#Building model
rf_md= RandomForestClassifier(n_estimators= 500, max_depth=5).fit(x_train_imp, y_train)

#Prediction on the test
rf_pred= rf_md.predict_proba(x_test_imp)[:,1]

#Changing likelyhoods to labels
rf_label= np.where(rf_pred < .1, 0, 1)

print(classification_report(y_test, rf_label))

              precision    recall  f1-score   support

           0       0.95      0.35      0.51       719
           1       0.20      0.89      0.32       129

    accuracy                           0.43       848
   macro avg       0.57      0.62      0.42       848
weighted avg       0.83      0.43      0.48       848



AdaBoost

In [21]:
#Building model
ada_md= AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth=5), n_estimators= 500, learning_rate=.01).fit(x_train_imp, y_train)

#Prediction on the test
ada_pred= ada_md.predict_proba(x_test_imp)[:,1]

#Changing likelyhoods to labels
ada_label= np.where(ada_pred < .1, 0, 1)

print(classification_report(y_test, ada_label))

              precision    recall  f1-score   support

           0       0.86      0.16      0.27       719
           1       0.15      0.85      0.26       129

    accuracy                           0.26       848
   macro avg       0.50      0.50      0.26       848
weighted avg       0.75      0.26      0.26       848



In [ ]:
#Based on my results, I would continue with the Random Forest model because it had an accuracy of .43 where AdaBoost had an accuracy of .26